In [ ]:
import yfinance as yf




In [ ]:
def download_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

stocks = ['BAJFINANCE.NS', 'HDFCAMC.NS', 'ASIANPAINT.NS', 'TCS.NS', 'DRREDDY.NS', '^NSEI']  # ^NSEI for Nifty 50
start_date = '2012-01-01'
end_date = '2023-01-26'

stock_data = {stock: download_stock_data(stock, start_date, end_date) for stock in stocks}

In [ ]:
pip install feedparser textblob


In [33]:
import feedparser
from textblob import TextBlob
import pandas as pd

In [34]:
def fetch_news(stock_name):
    feed_url = f'https://news.google.com/rss/search?q={stock_name}+when:7d&hl=en-IN&gl=IN&ceid=IN:en'
    news_feed = feedparser.parse(feed_url)
    return [(entry.title, entry.link, entry.published) for entry in news_feed.entries]

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

def get_stock_news_sentiment(stock_names):
    all_news_sentiments = []
    for stock in stock_names:
        news_items = fetch_news(stock)
        for title, link, published in news_items:
            sentiment_score = analyze_sentiment(title)  # Analyzing sentiment of the news title
            all_news_sentiments.append([stock, title, link, published, sentiment_score])
    return pd.DataFrame(all_news_sentiments, columns=['Stock', 'Title', 'Link', 'Published', 'Sentiment'])

In [35]:
# List of stocks
stocks = ['BAJFINANCE', 'HDFCAMC', 'ASIANPAINT', 'TCS', 'DRREDDY']

In [ ]:
# Get news sentiment
news_sentiment = get_stock_news_sentiment(stocks)
print(news_sentiment)

In [37]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
import numpy as np

In [54]:
def get_stock_data(ticker, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date)

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(0)

def calculate_sma(data, window=10):
    return data['Close'].rolling(window=window).mean()

def calculate_ema(data, window=10):
    return data['Close'].ewm(span=window, adjust=False).mean()

def perform_sentiment_analysis(text):
    return TextBlob(text).sentiment.polarity




In [55]:
tickers = ['BAJFINANCE.NS', 'HDFCAMC.NS', 'ASIANPAINT.NS', 'TCS.NS', 'DRREDDY.NS']
start_date = '2012-01-01'
end_date = '2023-12-26'

stock_data = {}
for ticker in tickers:
    df = get_stock_data(ticker, start_date, end_date)
    df['RSI'] = calculate_rsi(df)
    df['SMA'] = calculate_sma(df)
    df['EMA'] = calculate_ema(df)
    stock_data[ticker] = df



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [58]:
for ticker in stock_data:
    # Fetch news and analyze sentiment
    news_items = fetch_news(ticker)
    sentiment_scores = [analyze_sentiment(title) for title, _, _ in news_items]
    avg_sentiment_score = np.mean(sentiment_scores) if sentiment_scores else 0

    # Add Avg_Sentiment to the DataFrame
    stock_data[ticker]['Avg_Sentiment'] = avg_sentiment_score



In [63]:
#tweaked nifty data added
start_date = '2012-01-01'
end_date = '2023-12-26'
nifty_data = get_stock_data('^NSEI', start_date, end_date)  # '^NSEI' is the ticker for Nifty 50



[*********************100%%**********************]  1 of 1 completed


In [64]:
nifty_data['RSI_Nifty'] = calculate_rsi(nifty_data)
nifty_data['SMA_Nifty'] = calculate_sma(nifty_data)
nifty_data['EMA_Nifty'] = calculate_ema(nifty_data)


In [65]:
for ticker in stock_data:
    stock_data[ticker] = stock_data[ticker].merge(nifty_data[['RSI_Nifty', 'SMA_Nifty', 'EMA_Nifty']], left_index=True, right_index=True, how='left')


In [66]:
combined_data = pd.concat(stock_data.values())
scaler = StandardScaler()
features = ['RSI', 'SMA', 'EMA', 'Avg_Sentiment','RSI_Nifty', 'SMA_Nifty', 'EMA_Nifty']
combined_data[features] = scaler.fit_transform(combined_data[features])

combined_data['Target'] = combined_data['Close'].shift(-5) > combined_data['Close']
X = combined_data[features].fillna(0)
y = combined_data['Target'].fillna(False)


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

       False       0.75      0.67      0.71      1764
        True       0.75      0.82      0.78      2180

    accuracy                           0.75      3944
   macro avg       0.75      0.74      0.75      3944
weighted avg       0.75      0.75      0.75      3944

